### Ingestión del archivo "country.json"

In [0]:
dbutils.widgets.text("p_environment","")
v_environment = dbutils.widgets.get("p_environment")

In [0]:
dbutils.widgets.text("p_file_date","2024-12-16")
v_file_date = dbutils.widgets.get("p_file_date")

In [0]:
%run "../includes/configuration"

In [0]:
%run "../includes/common_functions"

#### Paso 1 - Leer archvio JSON usando "DataFrameReader" de Spark

In [0]:
countries_schema = "countryId INT, countryIsoCode STRING, countryName STRING"

In [0]:
countries_df = spark.read \
    .schema(countries_schema) \
    .json(f"{bronze_folder_path}/{v_file_date}/country.json")


#### Pase 2 - Eliminar columnas no deseadas del DataFrame

In [0]:
from pyspark.sql.functions import col

In [0]:
#countries_dropped_df = countries_df.drop("countryIsoCode")
#countries_dropped_df = countries_df.drop(countries_df["countryIsoCode"])
countries_dropped_df = countries_df.drop(col("countryIsoCode"))

#### Paso 3 - Cambiar el nombre de las columnas y agregar la columna "ingestion_date" y "environment" al data frame

In [0]:
from pyspark.sql.functions import current_timestamp, lit

In [0]:
countries_final_df = add_ingestion_date(countries_dropped_df) \
    .withColumnRenamed("countryId","country_id") \
    .withColumnRenamed("countryName","country_name") \
    .withColumn("environment", lit(v_environment)) \
    .withColumn("file_date", lit(v_file_date))
#countries_final_df = countries_dropped_df \
#    .withColumnRenamed("countryId","country_id") \
#    .withColumnRenamed("countryName","country_name") \
#    .withColumn("ingestion_date", current_timestamp()) \
#    .withColumn("environment", lit("production"))

#### Paso 4 - Escribir datos en el DataLake en formato "Parquet"

In [0]:
countries_final_df.write.mode("overwrite").format("delta").saveAsTable("movie_silver.countries")
# countries_final_df.write.mode("overwrite").format("parquet").saveAsTable("movie_silver.countries")
# countries_final_df.write.mode("overwrite").parquet(f"{silver_folder_path}/countries")

In [0]:
%sql
SELECT * FROM movie_silver.countries;

In [0]:
dbutils.notebook.exit("Success")